# Imports

In [21]:
import pandas as pd
from matplotlib import pyplot as plt
from geopy.geocoders import Nominatim

# Reading Data

In [7]:
pd.set_option('display.max_columns', 25)
df = pd.read_csv('C:/Users/Ayron/repos/Datasets/kc_house_data.csv')

# 1. Qual a média do preço de compra dos imóveis por "Nível"?


In [1]:
# Nível 0 => Preço entre $ 0 e $ 321.950
# Nível 1 => Preço entre $ 321.950 e $ 450.000
# Nível 2 => Preço entre $ 450.000 e $ 645.000
# Nível 3 => Acima $ 645.000

In [15]:
for i in range(len(df)):
    if (df.loc[i, 'price'] > 0) & (df.loc[i, 'price'] < 321950):
        df.loc[i, 'level'] = 0
    elif (df.loc[i, 'price'] >= 321950) & (df.loc[i, 'price'] < 450000):
        df.loc[i, 'level'] = 1
    elif (df.loc[i, 'price'] >= 450000) & (df.loc[i, 'price'] < 645000):
        df.loc[i, 'level'] = 2
    else:
        df.loc[i, 'level'] = 3

In [16]:
df[['price', 'level']].groupby('level').mean().reset_index()

,level,price
0,0,251544.621507
1,1,383585.205332
2,2,536164.422840
3,3,985008.983189


# 2. Qual a média dor tamanho da sala de estar dos imóveis por "Size"?
* Size 0 => Tamanho entre 0 e 1427 sqft
* Size 1 => Tamanho entre 1427 e 1910 sqft
* Size 2 => Tamanho entre 1910 e 2550 sqft
* Size 3 => Tamanho acima de 2550 sqft

In [17]:
for i in range(len(df)):
    if (df.loc[i, 'sqft_living'] > 0) & (df.loc[i, 'sqft_living'] < 1427):
        df.loc[i, 'size_living'] = 0
    elif (df.loc[i, 'sqft_living'] >= 1427) & (df.loc[i, 'sqft_living'] < 1910):
        df.loc[i, 'size_living'] = 1
    elif (df.loc[i, 'sqft_living'] >= 1910) & (df.loc[i, 'sqft_living'] < 2550):
        df.loc[i, 'size_living'] = 2
    else:
        df.loc[i, 'sqft_living'] = 3

In [18]:
df[['sqft_living', 'size_living']].groupby('size_living').mean().reset_index()

,size_living,sqft_living
0,0.0,1123.776421
1,1.0,1661.198088
2,2.0,2202.700239


# 3. Adicione as seguintes informações ao conjunto de dados original:
* Place ID: Identificação da localização
* OSM Type: Open Street Map Type
* Country: Nome do País
* Country Code: Código do País

In [ ]:
#Creaty empty rows
df['Place_ID'] = 'NA'
df['OSM_TypeID'] = 'NA'
df['Country'] = 'NA'
df['Country_Code'] = 'NA'

#Inicializando Nominatim API
geolocator = Nominatim(user_agent='geoapiExercises')

for i in range(len(df)):
    #Make query
    query = str(df.loc[i, 'lat']) + ',' + str(df.loc[i, 'long'])
    
    #API Request
    response = geolocator.reverse(query)



df.loc[0, ]